# Rains of Australia

## Загрузка и чтение данных

In [41]:
!pip install catboost

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.impute import SimpleImputer

In [43]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [44]:
df = pd.read_csv("/content/drive/MyDrive/weatherAUS.csv")

In [45]:
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


### Сводка данных о датасете

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

Некоторые признаки имеют категориальные значения, необходимо перевести их в числовые.

## Произведено кодирование категориальных признаков

In [47]:
label_encoder = LabelEncoder()

Удалим строки в которых отсутствует RainTomorrow

In [48]:
df.dropna(subset=['RainTomorrow'], inplace=True)

In [49]:
df['Date'] = pd.to_datetime(df['Date'])

In [50]:
df['Location'] = label_encoder.fit_transform(df['Location'])
df['WindGustDir'] = label_encoder.fit_transform(df['WindGustDir'].astype(str))
df['WindDir9am'] = label_encoder.fit_transform(df['WindDir9am'].astype(str))
df['WindDir3pm'] = label_encoder.fit_transform(df['WindDir3pm'].astype(str))
df['RainToday'] = label_encoder.fit_transform(df['RainToday'].astype(str))
df['RainTomorrow'] = label_encoder.fit_transform(df['RainTomorrow'].astype(str))

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142193 entries, 0 to 145458
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           142193 non-null  datetime64[ns]
 1   Location       142193 non-null  int64         
 2   MinTemp        141556 non-null  float64       
 3   MaxTemp        141871 non-null  float64       
 4   Rainfall       140787 non-null  float64       
 5   Evaporation    81350 non-null   float64       
 6   Sunshine       74377 non-null   float64       
 7   WindGustDir    142193 non-null  int64         
 8   WindGustSpeed  132923 non-null  float64       
 9   WindDir9am     142193 non-null  int64         
 10  WindDir3pm     142193 non-null  int64         
 11  WindSpeed9am   140845 non-null  float64       
 12  WindSpeed3pm   139563 non-null  float64       
 13  Humidity9am    140419 non-null  float64       
 14  Humidity3pm    138583 non-null  float64       
 15  Press

### Можно установить что в наборе данных имеются пустые значения, произведена замена пустых значений на средние

In [52]:
df.fillna(df.mean(), inplace=True)

In [53]:
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,2,13.4,22.9,0.6,5.469824,7.624853,13,44.0,13,...,71.0,22.0,1007.7,1007.1,8.000000,4.503167,16.9,21.8,0,0
1,2008-12-02,2,7.4,25.1,0.0,5.469824,7.624853,14,44.0,6,...,44.0,25.0,1010.6,1007.8,4.437189,4.503167,17.2,24.3,0,0
2,2008-12-03,2,12.9,25.7,0.0,5.469824,7.624853,15,46.0,13,...,38.0,30.0,1007.6,1008.7,4.437189,2.000000,21.0,23.2,0,0
3,2008-12-04,2,9.2,28.0,0.0,5.469824,7.624853,4,24.0,9,...,45.0,16.0,1017.6,1012.8,4.437189,4.503167,18.1,26.5,0,0
4,2008-12-05,2,17.5,32.3,1.0,5.469824,7.624853,13,41.0,1,...,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,2017-06-20,41,3.5,21.8,0.0,5.469824,7.624853,0,31.0,2,...,59.0,27.0,1024.7,1021.2,4.437189,4.503167,9.4,20.9,0,0
145455,2017-06-21,41,2.8,23.4,0.0,5.469824,7.624853,0,31.0,9,...,51.0,24.0,1024.6,1020.3,4.437189,4.503167,10.1,22.4,0,0
145456,2017-06-22,41,3.6,25.3,0.0,5.469824,7.624853,6,22.0,9,...,56.0,21.0,1023.5,1019.1,4.437189,4.503167,10.9,24.5,0,0
145457,2017-06-23,41,5.4,26.9,0.0,5.469824,7.624853,3,37.0,9,...,53.0,24.0,1021.0,1016.8,4.437189,4.503167,12.5,26.1,0,0


In [54]:
print(df['RainTomorrow'].value_counts())

RainTomorrow
0    110316
1     31877
Name: count, dtype: int64


### Разделение данных на признаки и целевую переменную

In [55]:
X = df.drop(columns=['RainTomorrow', 'Date'])
y = df['RainTomorrow']

## Модель knn


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Для приведения данных к одному масштабу произведено скалирование с помощью StandartScaler.

In [57]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [58]:
n_neighbors = 5

In [59]:
knn = KNeighborsClassifier(n_neighbors=n_neighbors)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [60]:
y_pred = knn.predict(X_test)

### Оценка модели

In [61]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8377580083687893


Таким образом, точность модели равна: 0.8377580083687893.

## Метод машин опорных векторов

In [ ]:
svm = SVC(kernel='rbf', C=1.0)
svm.fit(X_train, y_train)

SVC()

In [ ]:
y_pred = svm.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8140038498556305


Таким образом, точность модели равна: 0.8140038498556305

## Линейная и логистическая регрессия


### Линейная регрессия

In [ ]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

### Логистическая регрессия

In [ ]:
logistic_reg = LogisticRegression()
logistic_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Предсказания на тестовом наборе

In [ ]:
y_pred_linear = linear_reg.predict(X_test)
y_pred_logistic = logistic_reg.predict(X_test)

### Оценка модели

Для оценки модели линейной регрессии используем среднеквадратичную ошибку(MSE), для оценки логистической регрессии используем точность.

In [ ]:
mse_linear = mean_squared_error(y_test, y_pred_linear)
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)

print("Mean Squared Error (Linear Regression):", mse_linear)
print("Accuracy Logistic Regression:", accuracy_logistic)

Mean Squared Error (Linear Regression): 0.16881974407480338
Accuracy Logistic Regression: 0.8172349786882992


## Метод наивного Байеса

Импортируем классификатор MultinomialNB из модуля sklearn.naive_bayes. Этот классификатор реализует наивный Байесовский классификатор для данных. Гиперпараметр alpha контролирует регуляризацию модели

In [ ]:
nb_classifier = MultinomialNB(alpha=1.0)

In [ ]:
scaler = MinMaxScaler()
X_train_scaled_min_max = scaler.fit_transform(X_train)
X_test_scaled_min_max = scaler.transform(X_test)

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_scaled_min_max, y_train)

MultinomialNB()

Оценка производительности модели

In [ ]:
cv_scores = cross_val_score(nb_classifier, X_train_scaled_min_max, y_train, cv=5)

In [ ]:
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())

Cross-Validation Scores: [0.75943112 0.75938816 0.75938816 0.75929188 0.75963563]
Mean CV Score: 0.7594269908697481


## Метод решающего дерева и случайного леса

### Решающее дерево

Метод решающего дерева строит дерево решений на основе обучающих данных. Каждый узел дерева представляет собой условие на признаке, а каждое ребро представляет собой результат этого условия. В листовых узлах дерева содержатся предсказания



In [ ]:
dt_classifier = DecisionTreeClassifier(max_depth=5, min_samples_split=2, min_samples_leaf=1)
dt_cv_scores = cross_val_score(dt_classifier, X_train, y_train, cv=5)

### Случайный лес

Метод случайного леса  является ансамблевым методом, который объединяет несколько деревьев решений для классификации или регрессии. Каждое дерево строится на основе подвыборки данных и случайного подмножества признаков. Затем результаты всех деревьев усредняются (для регрессии) или принимается решение путем голосования (для классификации).

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_features='auto')
rf_cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=5)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

In [ ]:
print("Decision Tree CV Scores:", dt_cv_scores)
print("Decision Tree Mean CV Score:", dt_cv_scores.mean())
print("Random Forest CV Scores:", rf_cv_scores)
print("Random Forest Mean CV Score:", rf_cv_scores.mean())

Decision Tree CV Scores: [0.83058348 0.81936925 0.82435336 0.82563486 0.8232716 ]
Decision Tree Mean CV Score: 0.8246425101526006
Random Forest CV Scores: [0.85073473 0.84162585 0.84660995 0.84540025 0.84475573]
Random Forest Mean CV Score: 0.8458252999568165


## Метод CatBoost

CatBoost - это высокоэффективная библиотека градиентного бустинга, разработанная компанией Яндекс. Она специально оптимизирована для работы с категориальными признаками и автоматически обрабатывает их без необходимости предварительного кодирования.

Гиперпараметры: iterations: количество итераций обучения; learning_rate: скорость обучения; depth - максимальная глубина дерева; border_count: количество корзин для числовых признаков. Этот параметр влияет на скорость обучения и качество модели.

In [ ]:
catboost_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, border_count=32)
catboost_cv_scores = cross_val_score(catboost_classifier, X_train, y_train, cv=5)

0:	learn: 0.9868155	total: 95.7ms	remaining: 1m 35s
1:	learn: 0.8994384	total: 151ms	remaining: 1m 15s
2:	learn: 0.8308184	total: 196ms	remaining: 1m 5s
3:	learn: 0.7738326	total: 260ms	remaining: 1m 4s
4:	learn: 0.7280553	total: 310ms	remaining: 1m 1s
5:	learn: 0.6878259	total: 355ms	remaining: 58.8s
6:	learn: 0.6543525	total: 399ms	remaining: 56.6s
7:	learn: 0.6262831	total: 446ms	remaining: 55.3s
8:	learn: 0.6017769	total: 495ms	remaining: 54.5s
9:	learn: 0.5799840	total: 545ms	remaining: 54s
10:	learn: 0.5612221	total: 599ms	remaining: 53.9s
11:	learn: 0.5442103	total: 648ms	remaining: 53.4s
12:	learn: 0.5302821	total: 698ms	remaining: 53s
13:	learn: 0.5175044	total: 751ms	remaining: 52.9s
14:	learn: 0.5066935	total: 802ms	remaining: 52.6s
15:	learn: 0.4970721	total: 871ms	remaining: 53.5s
16:	learn: 0.4885927	total: 955ms	remaining: 55.2s
17:	learn: 0.4803604	total: 1.03s	remaining: 56.3s
18:	learn: 0.4731357	total: 1.13s	remaining: 58.3s
19:	learn: 0.4666347	total: 1.21s	remainin

In [ ]:
print("CatBoost CV Scores:", catboost_cv_scores)
print("CatBoost Mean CV Score:", catboost_cv_scores.mean())

CatBoost CV Scores: [0.8548595  0.84708258 0.85069176 0.85184549 0.85270485]
CatBoost Mean CV Score: 0.8514368355275183
